- checked on 231007 MJ SON

In [1]:
import pandas as pd
import numpy as np
import quantecon as qe
from scipy.stats import norm
from scipy.interpolate import interp1d
import math
import matplotlib.pyplot as plt
import scipy.integrate as integrate
# import ray
import time

## Parameter

In [2]:
beta = 0.96
alpha = 0.36
delta = 0.08

risk_aver_vec = [1,3,5]
rho_vec = [0.0, 0.3, 0.6, 0.9]
sigma_vec = [0.2, 0.4]

risk_aver = 3
rho = 0.6
sigma = 0.2

amin = 0.0
amax = 20
na = 150
naf = na*100
ne = 7
agrid = np.linspace(amin, amax, na)
afgrid = np.linspace(amin, amax, naf)

tol_iter = 1e-3

interp_kind = 'cubic'
# The string has to be one of 
#‘linear’, ‘nearest’, ‘nearest-up’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, or ‘next’.

In [3]:
# Markov chain
mc = qe.markov.approximation.tauchen(rho = rho, sigma_u = sigma*((1-rho**2)**(1/2)) , n=ne)
P = mc.P
stationary_dist = mc.stationary_distributions
stationary_dist

array([[0.00716548, 0.06402864, 0.24130663, 0.37499849, 0.24130663,
        0.06402864, 0.00716548]])

In [4]:
state_values = mc.state_values
state_values

array([-0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6])

In [5]:
labor_endowment = np.exp(mc.state_values)
labor_endowment

array([0.54881164, 0.67032005, 0.81873075, 1.        , 1.22140276,
       1.4918247 , 1.8221188 ])

In [6]:
egrid = labor_endowment

## Function

In [7]:
def u(c):
    if risk_aver == 1:
        u = np.log(c)
    else:
        u = (c**(1-risk_aver)-1)/(1-risk_aver) 
    
    return u

In [8]:
def wage(k,l=1):
    wage = (1-alpha) * k**alpha * l**(-alpha)
    return wage

def interestrate(k, l=1):
    interest = alpha * k**(alpha-1) * l**(1-alpha) - delta
    return interest

def f(k,l):
    return k**alpha * l**(1-alpha)

In [9]:
# @ray.remote
def vfi(r,w, Vguess = np.ones((na, ne))):
    Vdiff_list =[] # Vdiff 확인 위해 임시

    V = Vguess.copy()

    Vdiff = 1
    Iter = 0

    while Vdiff > tol_iter:
        Iter = Iter +1
        Vlast = V.copy()
        
        # Vlast를 afgrid에 맞춰주기 위해서 보간법 사용
        temp1 = interp1d(agrid, Vlast[:,0], kind = interp_kind)(afgrid)
        temp2 = interp1d(agrid, Vlast[:,1], kind = interp_kind)(afgrid)
        temp3 = interp1d(agrid, Vlast[:,2], kind = interp_kind)(afgrid)
        temp4 = interp1d(agrid, Vlast[:,3], kind = interp_kind)(afgrid)
        temp5 = interp1d(agrid, Vlast[:,4], kind = interp_kind)(afgrid)
        temp6 = interp1d(agrid, Vlast[:,5], kind = interp_kind)(afgrid)
        temp7 = interp1d(agrid, Vlast[:,6], kind = interp_kind)(afgrid)

        Vlast_interp = np.array([temp1, temp2, temp3, temp4, temp5, temp6, temp7]).T
        
        # pre-allocation
        V = np.zeros((na, ne))
        sav = np.zeros((na, ne))
        savind = np.zeros((na, ne), dtype = int)
        con = np.zeros((na, ne))
        
        # Loop over assets
        for ia in range(0, na):
            
            # Loop over endowments
            for ie in range(0,ne):
                income = (1+r)*agrid[ia] + w*labor_endowment[ie] 
                # 현재 endowment가 el인지 eh인지에 따라서 다음기 확률이 달라지므로
                
                Vchoice = u(np.maximum(income - afgrid.T, 1.0e-10)) + beta*(Vlast_interp @ P.T)[:,ie]
                V[ia, ie] = np.max(Vchoice)
                savind[ia, ie] = np.argmax(Vchoice)
                sav[ia, ie] = afgrid[savind[ia, ie]]
                con[ia, ie] = income - sav[ia, ie]

        # Value function의 값이 수렴하는지 확인        
        Vdiff = np.max(np.max(abs(V-Vlast)))
        Vdiff_list.append(Vdiff)
        if Display == 1 :
            print('Iteration no. ' + str(Iter), ' max val fn diff is ' + str(Vdiff))     
            
    return V, Vlast, Vdiff_list, savind, sav, con

In [21]:
temptemp = np.array([1,2,3,4,5])

In [32]:
temptemp[temptemp == 5] = 4

In [33]:
temptemp

array([1, 2, 3, 4, 4])

In [45]:
# @ray.remote
def simulate(Nsim = 50000, Tsim = 500):
    
    # saving interpolation: afgrid에서 afgrid로 매핑하는 건 없기 때문에 만들어주려고
    tem1 = interp1d(agrid, sav[:,0], kind=interp_kind)(afgrid)
    tem2 = interp1d(agrid, sav[:,1], kind=interp_kind)(afgrid)
    tem3 = interp1d(agrid, sav[:,2], kind=interp_kind)(afgrid)
    tem4 = interp1d(agrid, sav[:,3], kind=interp_kind)(afgrid)
    tem5 = interp1d(agrid, sav[:,4], kind=interp_kind)(afgrid)
    tem6 = interp1d(agrid, sav[:,5], kind=interp_kind)(afgrid)
    tem7 = interp1d(agrid, sav[:,6], kind=interp_kind)(afgrid)

    sav_interp = np.array([tem1, tem2, tem3, tem4, tem5, tem6, tem7]).T

    t1 = interp1d(agrid, savind[:,0], kind=interp_kind)(afgrid)
    t2 = interp1d(agrid, savind[:,1], kind=interp_kind)(afgrid)
    t3 = interp1d(agrid, savind[:,2], kind=interp_kind)(afgrid)
    t4 = interp1d(agrid, savind[:,3], kind=interp_kind)(afgrid)
    t5 = interp1d(agrid, savind[:,4], kind=interp_kind)(afgrid)
    t6 = interp1d(agrid, savind[:,5], kind=interp_kind)(afgrid)
    t7 = interp1d(agrid, savind[:,6], kind=interp_kind)(afgrid)

    savind_interp = np.array([t1, t2, t3, t4, t5, t6, t7]).T
    savind_interp = np.around(savind_interp).astype('int')
    savind_interp[savind_interp >= naf] = naf-1 # 인덱스 반올림해서 인덱스 범위 벗어난 경우를 방지하고자 
    
    # pre-allocation
    esim = np.zeros((Nsim, Tsim)) 
    aindsim = np.zeros((Nsim, Tsim), dtype = int) # ind = index

    # Initial asset allocation
    aindsim[:,0] = 0

    # Endowment allocation (Exogenous) 
    for i in range(0, Nsim):
        esim[i,:] = mc.simulate(Tsim).astype('float')
    
    esim = np.exp(esim)

    ### Loop over time periods

    for it in range(0, Tsim): # Tsim
        if Display == 1 & (it+1)%100 == 0: # time period 100 넘길 때마다 알려줌
            print(' Simulating, time period ' + str(it+1))

        # asset choice
        if it < Tsim-1:
            for ie in range(0, ne):
                e = egrid[ie]
                aindsim[esim[:,it] == e, it+1] = savind_interp[aindsim[esim[:,it] == e,it], ie] 

    asim = afgrid[aindsim]
    avg_asset = np.mean(asim[:,-1])
    
    return asim, esim, avg_asset

In [46]:
import scipy.integrate as integrate

def lorenz_and_gini(dist_wealth, agrid, curve=True):
    dist_wealth_normalize = dist_wealth / (dist_wealth.sum())
    
    agg_asset = dist_wealth_normalize @ agrid
    
    wealth_percentile = []
    for i in range(len(agrid)+1):
        value = (dist_wealth_normalize[:i] @ agrid[:i]) / agg_asset
        wealth_percentile.append(value)
    
    if dist_wealth_normalize.cumsum()[0] < 1e-20 :
        pop_cumsum = dist_wealth_normalize.cumsum()
    else:
        pop_cumsum = np.insert(dist_wealth_normalize.cumsum(), 0, 0)
        
    if wealth_percentile[0] < 1e-20:
        wealth_cumsum = wealth_percentile
    else:
        wealth_cumsum = np.insert(wealth_percentile, 0, 0)
    
    if curve == True:
        plt.plot(pop_cumsum, wealth_cumsum)
        plt.plot(wealth_cumsum, wealth_cumsum)
        plt.title('Lorenz curve')
        plt.xlabel('population')
        plt.ylabel('wealth')
        plt.ylim(0,1)
        plt.xlim(0,1)
        plt.grid(True)
    else: 
        pass

    ### Gini coefficient
    lorenz = interp1d(pop_cumsum, wealth_cumsum)
    gini = (0.5 - integrate.quad(lorenz, 0, 1)[0])/0.5
    print(f'gini = {gini}')
    
    return gini

## Equilibrium r 찾기

In [47]:
1/beta - 1

0.04166666666666674

In [48]:
delta

0.08

In [49]:
# assume delta = 0
Display = 0 
phi = 0.8 
iternum = 0
crit = 1
crit_list = []

r_old = 2
r = 0.03

while iternum < 1000 and crit > 1e-2:
    iternum += 1
    crit = abs(r - r_old)/r_old
    crit_list.append(crit)

    k = (r/alpha)**(1/(alpha-1)) # * l 인데 (l = 1)
    w = r*(1-alpha)*k / alpha # / (alpha*L)인데 l = 1

    print(f'iter_num = {iternum}, r = {r}, k = {k}, crit = {crit}')
    
    ## VFI
#     ray.shutdown()
#     ray.init(num_cpus = 8)    
    V, Vlast, Vdiff_list, savind, sav, con =  vfi(r,w) # ray.get(vfi.remote(r))
#     ray.shutdown()
    
    ## Simulation
#     ray.init(num_cpus=8)
    asim, esim, avg_asset =  simulate() #ray.get(simulate.remote())

    k_old = k
    knew = avg_asset
    
    k = k_old*phi + knew*(1-phi)
    
    rnew = alpha* k**(alpha-1) # *L**(1-alpha)인데 l = 1
    
    r_old = r
    
    r = r_old*phi + rnew*(1-phi)
    
    V_guess = V # 이전 r 값에서 수렴시킨 value function 값을 guess로 사용하면 속도가 더 빨라질 수 있으니까


iter_num = 1, r = 0.03, k = 48.55351735917101, crit = 0.985
 Simulating, time period 2
 Simulating, time period 4
 Simulating, time period 6
 Simulating, time period 8
 Simulating, time period 10
 Simulating, time period 12
 Simulating, time period 14
 Simulating, time period 16
 Simulating, time period 18
 Simulating, time period 20
 Simulating, time period 22
 Simulating, time period 24
 Simulating, time period 26
 Simulating, time period 28
 Simulating, time period 30
 Simulating, time period 32
 Simulating, time period 34
 Simulating, time period 36
 Simulating, time period 38
 Simulating, time period 40
 Simulating, time period 42
 Simulating, time period 44
 Simulating, time period 46
 Simulating, time period 48
 Simulating, time period 50
 Simulating, time period 52
 Simulating, time period 54
 Simulating, time period 56
 Simulating, time period 58
 Simulating, time period 60
 Simulating, time period 62
 Simulating, time period 64
 Simulating, time period 66
 Simulating, time pe

 Simulating, time period 74
 Simulating, time period 76
 Simulating, time period 78
 Simulating, time period 80
 Simulating, time period 82
 Simulating, time period 84
 Simulating, time period 86
 Simulating, time period 88
 Simulating, time period 90
 Simulating, time period 92
 Simulating, time period 94
 Simulating, time period 96
 Simulating, time period 98
 Simulating, time period 100
 Simulating, time period 102
 Simulating, time period 104
 Simulating, time period 106
 Simulating, time period 108
 Simulating, time period 110
 Simulating, time period 112
 Simulating, time period 114
 Simulating, time period 116
 Simulating, time period 118
 Simulating, time period 120
 Simulating, time period 122
 Simulating, time period 124
 Simulating, time period 126
 Simulating, time period 128
 Simulating, time period 130
 Simulating, time period 132
 Simulating, time period 134
 Simulating, time period 136
 Simulating, time period 138
 Simulating, time period 140
 Simulating, time period 14

 Simulating, time period 138
 Simulating, time period 140
 Simulating, time period 142
 Simulating, time period 144
 Simulating, time period 146
 Simulating, time period 148
 Simulating, time period 150
 Simulating, time period 152
 Simulating, time period 154
 Simulating, time period 156
 Simulating, time period 158
 Simulating, time period 160
 Simulating, time period 162
 Simulating, time period 164
 Simulating, time period 166
 Simulating, time period 168
 Simulating, time period 170
 Simulating, time period 172
 Simulating, time period 174
 Simulating, time period 176
 Simulating, time period 178
 Simulating, time period 180
 Simulating, time period 182
 Simulating, time period 184
 Simulating, time period 186
 Simulating, time period 188
 Simulating, time period 190
 Simulating, time period 192
 Simulating, time period 194
 Simulating, time period 196
 Simulating, time period 198
 Simulating, time period 200
 Simulating, time period 202
 Simulating, time period 204
 Simulating, t

 Simulating, time period 218
 Simulating, time period 220
 Simulating, time period 222
 Simulating, time period 224
 Simulating, time period 226
 Simulating, time period 228
 Simulating, time period 230
 Simulating, time period 232
 Simulating, time period 234
 Simulating, time period 236
 Simulating, time period 238
 Simulating, time period 240
 Simulating, time period 242
 Simulating, time period 244
 Simulating, time period 246
 Simulating, time period 248
 Simulating, time period 250
 Simulating, time period 252
 Simulating, time period 254
 Simulating, time period 256
 Simulating, time period 258
 Simulating, time period 260
 Simulating, time period 262
 Simulating, time period 264
 Simulating, time period 266
 Simulating, time period 268
 Simulating, time period 270
 Simulating, time period 272
 Simulating, time period 274
 Simulating, time period 276
 Simulating, time period 278
 Simulating, time period 280
 Simulating, time period 282
 Simulating, time period 284
 Simulating, t

 Simulating, time period 286
 Simulating, time period 288
 Simulating, time period 290
 Simulating, time period 292
 Simulating, time period 294
 Simulating, time period 296
 Simulating, time period 298
 Simulating, time period 300
 Simulating, time period 302
 Simulating, time period 304
 Simulating, time period 306
 Simulating, time period 308
 Simulating, time period 310
 Simulating, time period 312
 Simulating, time period 314
 Simulating, time period 316
 Simulating, time period 318
 Simulating, time period 320
 Simulating, time period 322
 Simulating, time period 324
 Simulating, time period 326
 Simulating, time period 328
 Simulating, time period 330
 Simulating, time period 332
 Simulating, time period 334
 Simulating, time period 336
 Simulating, time period 338
 Simulating, time period 340
 Simulating, time period 342
 Simulating, time period 344
 Simulating, time period 346
 Simulating, time period 348
 Simulating, time period 350
 Simulating, time period 352
 Simulating, t

...

 Simulating, time period 264
 Simulating, time period 266
 Simulating, time period 268
 Simulating, time period 270
 Simulating, time period 272
 Simulating, time period 274
 Simulating, time period 276
 Simulating, time period 278
 Simulating, time period 280
 Simulating, time period 282
 Simulating, time period 284
 Simulating, time period 286
 Simulating, time period 288
 Simulating, time period 290
 Simulating, time period 292
 Simulating, time period 294
 Simulating, time period 296
 Simulating, time period 298
 Simulating, time period 300
 Simulating, time period 302
 Simulating, time period 304
 Simulating, time period 306
 Simulating, time period 308
 Simulating, time period 310
 Simulating, time period 312
 Simulating, time period 314
 Simulating, time period 316
 Simulating, time period 318
 Simulating, time period 320
 Simulating, time period 322
 Simulating, time period 324
 Simulating, time period 326
 Simulating, time period 328
 Simulating, time period 330
 Simulating, t

 Simulating, time period 348
 Simulating, time period 350
 Simulating, time period 352
 Simulating, time period 354
 Simulating, time period 356
 Simulating, time period 358
 Simulating, time period 360
 Simulating, time period 362
 Simulating, time period 364
 Simulating, time period 366
 Simulating, time period 368
 Simulating, time period 370
 Simulating, time period 372
 Simulating, time period 374
 Simulating, time period 376
 Simulating, time period 378
 Simulating, time period 380
 Simulating, time period 382
 Simulating, time period 384
 Simulating, time period 386
 Simulating, time period 388
 Simulating, time period 390
 Simulating, time period 392
 Simulating, time period 394
 Simulating, time period 396
 Simulating, time period 398
 Simulating, time period 400
 Simulating, time period 402
 Simulating, time period 404
 Simulating, time period 406
 Simulating, time period 408
 Simulating, time period 410
 Simulating, time period 412
 Simulating, time period 414
 Simulating, t

 Simulating, time period 412
 Simulating, time period 414
 Simulating, time period 416
 Simulating, time period 418
 Simulating, time period 420
 Simulating, time period 422
 Simulating, time period 424
 Simulating, time period 426
 Simulating, time period 428
 Simulating, time period 430
 Simulating, time period 432
 Simulating, time period 434
 Simulating, time period 436
 Simulating, time period 438
 Simulating, time period 440
 Simulating, time period 442
 Simulating, time period 444
 Simulating, time period 446
 Simulating, time period 448
 Simulating, time period 450
 Simulating, time period 452
 Simulating, time period 454
 Simulating, time period 456
 Simulating, time period 458
 Simulating, time period 460
 Simulating, time period 462
 Simulating, time period 464
 Simulating, time period 466
 Simulating, time period 468
 Simulating, time period 470
 Simulating, time period 472
 Simulating, time period 474
 Simulating, time period 476
 Simulating, time period 478
 Simulating, t

 Simulating, time period 498
 Simulating, time period 500
iter_num = 65, r = 0.053938287064403456, k = 19.41481962698319, crit = 0.001135393654505856
 Simulating, time period 2
 Simulating, time period 4
 Simulating, time period 6
 Simulating, time period 8
 Simulating, time period 10
 Simulating, time period 12
 Simulating, time period 14
 Simulating, time period 16
 Simulating, time period 18
 Simulating, time period 20
 Simulating, time period 22
 Simulating, time period 24
 Simulating, time period 26
 Simulating, time period 28
 Simulating, time period 30
 Simulating, time period 32
 Simulating, time period 34
 Simulating, time period 36
 Simulating, time period 38
 Simulating, time period 40
 Simulating, time period 42
 Simulating, time period 44
 Simulating, time period 46
 Simulating, time period 48
 Simulating, time period 50
 Simulating, time period 52
 Simulating, time period 54
 Simulating, time period 56
 Simulating, time period 58
 Simulating, time period 60
 Simulating, t

KeyboardInterrupt: 

## Plot

In [ ]:
# Convergence of Value function
plt.plot(Vdiff_list)

In [ ]:
## Optimal Decision (saving = capital accumulation)
fig, ax = plt.subplots()
plt.plot(agrid, sav[:,0], label = 'lowest endowment')
plt.plot(agrid, sav[:,6], label = 'highest endowment')
plt.plot(agrid, agrid, label = '$45\,^{\circ}$')
plt.title('Optimal Saving Decision')
plt.xlabel('a')
plt.ylabel('a\'')
fig.legend()

In [ ]:
# Consumption Policy Function
fig, ax = plt.subplots()
plt.plot(agrid, con[:,0], label = 'lowest endowment')
plt.plot(agrid, con[:,6], label = 'highest endowment')
plt.title('Consumption Policy Function')
plt.xlabel('a')
plt.ylabel('Consumption')
fig.legend()

In [ ]:
[0.54881164, 0.67032005, 0.81873075, 1. , 1.22140276, 1.4918247 , 1.8221188 ]

In [ ]:
stationary_dist

In [ ]:
# endowment distribution
Tsim = 500
plt.hist(esim[:,Tsim-1] ,bins = len(egrid), edgecolor='black')
plt.ylabel('population')
plt.title('Labor endowment distribution')

In [ ]:
# asset distribution
plt.hist(asim[:,-1],bins = 40,facecolor=(.7,.7,.7),edgecolor='black')
plt.xlabel('asset level')
plt.ylabel('population')
plt.title('Asset distribution')

In [ ]:
asset_level = np.unique(asim[:,-1], return_counts = True)[0]

In [ ]:
pop_dist = np.unique(asim[:,-1], return_counts = True)[1]

In [ ]:
lorenz_and_gini(pop_dist, asset_level)